In [1]:
import sys
if 'google.colab' in sys.modules:
    print('Running in Colab.')
    !git clone https://github.com/YuanGongND/ast
    sys.path.append('./ast')
%cd /content/ast/

! pip install timm==0.4.5
! pip install wget
import librosa
import os, csv, argparse, wget
os.environ['TORCH_HOME'] = '/content/ast/pretrained_models'
if os.path.exists('/content/ast/pretrained_models') == False:
  os.mkdir('/content/ast/pretrained_models')
import torch, torchaudio, timm
import numpy as np
from torch.cuda.amp import autocast
import IPython

Running in Colab.
Cloning into 'ast'...
remote: Enumerating objects: 649, done.
remote: Counting objects: 100% (649/649), done.
remote: Compressing objects: 100% (293/293), done.
remote: Total 649 (delta 363), reused 599 (delta 347), pack-reused 0
Receiving objects: 100% (649/649), 2.40 MiB | 5.92 MiB/s, done.
Resolving deltas: 100% (363/363), done.
/content/ast
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29

In [2]:
import torch
if torch.cuda.is_available():
  print('Good to go!')
else:
  print('Please set GPU via Edit -> Notebook Settings.')
device = torch.device('cuda:0')

Good to go!


In [6]:
# reset completely
%cd /content/ast/
!rm -r data

/content/ast


In [7]:
import numpy as np
import json
import os
import wget

print('getting prep & run files')
if os.path.exists('./data') == False:
    os.mkdir('./data')
if os.path.exists('./data/GTZAN') == False:
    os.mkdir('./data/GTZAN')
    run_url = 'https://www.dropbox.com/scl/fi/gpftg37abkl9qtgya25yc/run_GTZAN_4_6_0320.sh?rlkey=7nm60l2y3oktcb65kmt8pichp&dl=1'
    print('Downloading the run .sh file')
    wget.download(run_url, out='./data/')
    print('Download complete')
    prep_url = 'https://www.dropbox.com/scl/fi/l60nggx1bshlvaeo1zpnc/prep_GTZAN.py?rlkey=esq7hlm0prr1mr2y74ijnusua&dl=1'
    print('Downloading the prep.py file')
    wget.download(prep_url, out='./data/')
    print('Download complete')
else:
    print('data folder already exists')
print('Acquiring prep & run finished')

getting prep & run files
Download complete
Download complete
Acquiring prep & run finished


In [4]:
## moved it into GTZAN tarball as prep_GTZAN.py -- do not run this

##Adapted from Yuan Gong (2021) -- yuangong@mit.edu

import numpy as np
import json
import os
import wget
# from google.colab import drive
# drive.mount('/content/drive')
# PATH_OF_DATA= '/content/gdrive/MyDrive/GTZAN/Data/'



# prepare the data of the GTZAN dataset.
print('Now download and process GTZAN dataset, it will take a few moments...')
# download the speechcommands dataset
if os.path.exists('./data/GTZAN') == True:
    # we use the 35 class v2 dataset, which is used in torchaudio https://pytorch.org/audio/stable/_modules/torchaudio/datasets/speechcommands.html
    sc_url = 'https://www.dropbox.com/scl/fi/04492o3tao21kbme2qhz7/GTZAN.tar.gz?rlkey=8zrwhr8dizhrkct796afg1i8m&dl=1'
    wget.download(sc_url, out='./data/')
    print('Finished Download, Extracting')

    os.system('tar -xzvf ./data/GTZAN.tar.gz -C ./data/GTZAN')
    #os.remove('./data/GTZAN.tar.gz')
# Use training list from coreyker - "https://github.com/coreyker/dnn-mgr/tree/master/gtzan"
with open('./data/GTZAN/Data/train_filtered.txt', 'r') as f:
  train_list = f.readlines()
with open('./data/GTZAN/Data/valid_filtered.txt', 'r') as f:
  val_list = f.readlines()
with open('./data/GTZAN/Data/test_filtered.txt', 'r') as f:
  test_list = f.readlines()

label_set = np.loadtxt('./data/GTZAN/Data/GTZAN_class_labels_indices.csv', delimiter=',', dtype='str')
label_map = {}
for i in range(1, len(label_set)):
    label_map[eval(label_set[i][2])] = label_set[i][0]
print(label_map)

# generate  json files
if os.path.exists('./data/datafiles') == False:
    os.mkdir('./data/datafiles')
    base_path = './data/GTZAN/Data/'
    for split in ['test_filtered', 'valid_filtered', 'train_filtered']:
        wav_list = []
        with open(base_path+split+'.txt', 'r') as f:
            filelist = f.readlines()
        for file in filelist:
            cur_label = label_map[file.split('/')[0]]
            cur_path = os.path.abspath(os.getcwd()) + '/data/GTZAN/Data/' + file.strip()
            cur_dict = {"wav": cur_path, "labels": '/m/spcmd'+cur_label.zfill(2)}
            wav_list.append(cur_dict)
        if split == 'train_filtered':
            with open('./data/datafiles/GTZAN_train_data.json', 'w') as f:
                json.dump({'data': wav_list}, f, indent=1)
        if split == 'test_filtered':
            with open('./data/datafiles/GTZAN_eval_data.json', 'w') as f:
                json.dump({'data': wav_list}, f, indent=1)
        if split == 'valid_filtered':
            with open('./data/datafiles/GTZAN_valid_data.json', 'w') as f:
                json.dump({'data': wav_list}, f, indent=1)
        print(split + ' data processing finished, total {:d} samples'.format(len(wav_list)))

    print('GTZAN dataset processing finished.')




Now download and process GTZAN dataset, it will take a few moments...


KeyboardInterrupt: 

In [ ]:
!pwd

/content/ast


In [8]:
%cd data

/content/ast/data


In [ ]:
# reset experiment w/o redownloading dataset
!rm -r exp

In [10]:
# Run current sh file to begin training and evaluation
# using ImageNet pretrained model for now since AudioSet pretrained gives error -- debug later
#PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

# Results stored in Log.txt, results.csv, and other files.
!chmod 755 run_GTZAN_4_6_0320.sh
!./run_GTZAN_4_6_0320.sh

+ export TORCH_HOME=../pretrained_models
+ TORCH_HOME=../pretrained_models
+ model=ast
+ dataset=GTZAN
+ imagenetpretrain=True
+ audiosetpretrain=False
+ bal=none
+ lr=2.5e-5
+ epoch=10
+ freqm=48
+ timem=48
+ mixup=0.6
+ batch_size=4
+ fstride=10
+ tstride=10
+ dataset_mean=-6.845978
+ dataset_std=5.5654526
+ audio_length=1500
+ noise=True
+ metrics=acc
+ loss=BCE
+ warmup=False
+ lrscheduler_start=5
+ lrscheduler_step=1
+ lrscheduler_decay=0.85
+ tr_data=./data/datafiles/GTZAN_train_data.json
+ val_data=./data/datafiles/GTZAN_valid_data.json
+ eval_data=./data/datafiles/GTZAN_eval_data.json
+ exp_dir=./exp/test-GTZAN-f10-t10-pTrue-b4-lr2.5e-5-decoupe
+ '[' -d ./exp/test-GTZAN-f10-t10-pTrue-b4-lr2.5e-5-decoupe ']'
+ mkdir -p ./exp/test-GTZAN-f10-t10-pTrue-b4-lr2.5e-5-decoupe
+ python ./prep_GTZAN.py
Now download and process GTZAN dataset, it will take a few moments...
Finished Download, Extracting
Data/
Data/pop/
Data/.DS_Store
Data/features_30_sec.csv
Data/run_GTZAN_4_5_1139pm.sh
Dat